# Constrained optimization

Now we will move to studying constrained optimizaton problems i.e., the full problem
$$
\begin{align} \
\min \quad &f(x)\\
\text{s.t.} \quad & g_j(x) \geq 0\text{ for all }j=1,\ldots,J\\
& h_k(x) = 0\text{ for all }k=1,\ldots,K\\
&a_i\leq x_i\leq b_i\text{ for all } i=1,\ldots,n\\
&x\in \mathbb R^n,
\end{align}
$$
where for all $i=1,\ldots,n$ it holds that $a_i,b_i\in \mathbb R$ or they may also be $-\infty$ of $\infty$.

For example, we can have an optimization problem
$$
\begin{align} \
\min \quad &x_1^2+x_2^2\\
\text{s.t.} \quad & x_1+x_2-1\geq 0\\
&-1\leq x_1\leq 1, x_2\leq 3.\\
\end{align}
$$

In order to optimize that problem, we can define the following python function:

In [13]:
import numpy as np
def f_constrained(x):
    return np.linalg.norm(x),[x[0]+x[1]-1],[]

Now, we can call the function:

In [24]:
(f_val,ieq,eq) = f_constrained([0,0])
print "Value of f is "+str(f_val)
if len(ieq)>0:
    print "The values of inequality constraints are:"
    for ieq_j in ieq:
        print str(ieq_j)+", "
if len(eq)>0:
    print "The values of the equality constraints are:"
    for eq_k in eq:
        print str(eq_k)+", "

Value of f is 0.0
The values of inequality constraints are:
-1, 


Is this solution feasible?

In [25]:
if all([ieq_j>=0 for ieq_j in ieq]) and all([eq_k==0 for eq_k in eq]):
    print "Solution is feasible"
else:
    print "Solution is infeasible"

Solution is infeasible


# Indirect and direct methods for constrained optimization

There are two categories of methods for constrained optimization: Indirect and direct methods. The main difference is that
1. Indirect methods convert the constrained optimization problem into a single or a sequence of unconstrained optimization problems, that are then solved. Often, the intermediate solutions do not need to be feasbiel, the sequence of solutions converges to a solution that is optimal (and, thus, feasible).
2. Direct methods deal with the constrained optimization problem directly. In this case, the intermediate solutions are feasible.

# Indirect methods

## Penalty function methods

**IDEA:** Include constraints into the objective function with the help of penalty functions that penalize constraint violations.

Let, $\alpha(x):\mathbb R^n\to\mathbb R$ be a function so that 
* $\alpha(x)=$ for all feasible $x$
* $\alpha(x)>0$ for all infeasible $x$.

Define optimization problems
$$
\begin{align} \
\min \qquad &f(x)+r\alpha(x)\\
\text{s.t.} \qquad &x\in \mathbb R^n
\end{align}
$$
for $r>0$ and $x_r$ be the optimal solutions of these problems.

In this case, the optimal solutions $x_r$ converge to the optimal solution of the constrained problem, when $r\to\infty$, if such solution exists.

For example, good ideas for penalty functions are
* $h_k(x)^2$ for equality constraints,
* $\left(\max\{0,g_j(x)\}\right)^2$ for inequality constraints.

In [73]:
def alpha(x,f):
    (_,ieq,eq) = f(x)
    return sum([min([0,ieq_j])**2 for ieq_j in ieq])+sum([eq_k**2 for eq_k in eq])

In [74]:
alpha([0,2],f_constrained)

0

In [75]:
def penalized_function(x,f,r):
    return f(x)[0] + r*alpha(x,f)

In [77]:
penalized_function([0,0],f_constrained,100)

100.0

In [172]:
from scipy.optimize import minimize
res = minimize(lambda x:penalized_function(x,f_constrained,100),[0,0],method='Nelder-Mead', 
         options={'disp': True})
print res.x

Optimization terminated successfully.
         Current function value: 0.705857
         Iterations: 49
         Function evaluations: 80
[ 0.49823437  0.49823438]


In [173]:
(f_val,ieq,eq) = f_constrained(res.x)
print "Value of f is "+str(f_val)
if len(ieq)>0:
    print "The values of inequality constraints are:"
    for ieq_j in ieq:
        print str(ieq_j)+", "
if len(eq)>0:
    print "The values of the equality constraints are:"
    for eq_k in eq:
        print str(eq_k)+", "

if all([ieq_j>=0 for ieq_j in ieq]) and all([eq_k==0 for eq_k in eq]):
    print "Solution is feasible"
else:
    print "Solution is infeasible"

Value of f is 0.704609810365
The values of inequality constraints are:
-0.00353125, 
Solution is infeasible


# Barrier function methods

**IDEA:** Prevent leaving the feasible region so that the value of the objective is $\infty$ outside the feasible set.

This method is only applicable to problems with inequality constraints and for which the set 
$$\{x\in \mathbb R^n: g_j(x)>0\text{ for all }j=1,\ldots,J\}$$
is non-empty.

Let $\beta:\{x\in \mathbb R^n: g_j(x)>0\text{ for all }j=1,\ldots,J\}\to \mathbb R$ be a function so that $\beta(x)\to \infty$, when $x\to\partial\{x\in \mathbb R^n: g_j(x)>0\text{ for all }j=1,\ldots,J\}$, where $\partial A$ is the boundary of the set $A$. Now, define optimization problem 
$$
\begin{align}
\min \qquad & f(x) + r\beta(x)\\
\text{s.t. } \qquad & x\in \{x\in \mathbb R^n: g_j(x)>0\text{ for all }j=1,\ldots,J\}.
\end{align}
$$
and let $x_r$ be the optimal solution of this problem (which we assume to exist for all $r>0$).

In this case, $x_r$ converges to the optimal solution of the problem (if it exists), when $r\to 0^+$ (i.e., $r$ converges to zero from the right).

A good idea for barrier algorithm is $\frac1{g_j(x)}$.

In [169]:
def beta(x,f):
    _,ieq,_ = f(x)
    try:
        value=sum([1/max([0,ieq_j]) for ieq_j in ieq])
    except ZeroDivisionError:
        value = float("inf")
    return value

In [170]:
def function_with_barrier(x,f,r):
    return f(x)[0]+r*beta(x,f)

In [171]:
from scipy.optimize import minimize
res = minimize(lambda x:function_with_barrier(x,f_constrained,0.001),[1,1],method='Nelder-Mead', 
         options={'disp': True})
print res.x

Optimization terminated successfully.
         Current function value: 0.760290
         Iterations: 38
         Function evaluations: 67
[ 0.51881766  0.51877295]


In [162]:
(f_val,ieq,eq) = f_constrained(res.x)
print "Value of f is "+str(f_val)
if len(ieq)>0:
    print "The values of inequality constraints are:"
    for ieq_j in ieq:
        print str(ieq_j)+", "
if len(eq)>0:
    print "The values of the equality constraints are:"
    for eq_k in eq:
        print str(eq_k)+", "
if all([ieq_j>=0 for ieq_j in ieq]) and all([eq_k==0 for eq_k in eq]):
    print "Solution is feasible"
else:
    print "Solution is infeasible"

Value of f is 0.733687357744
The values of inequality constraints are:
0.0375906108995, 
Solution is feasible
